# Spam Filter Program - Naive Bayes Classifier
We want to build a program that classifies messages as Spam or Non-Spam. To do this we will use a naive bayes classification algorithm. The dataset we will use to train the algorithm was put together by Jose Hidalgo and is hosted at the UCI Machine Learning Repository. It contains 5,572 SMS messages that have already been classified by humans.

## Exploring the dataset

In [1]:
# import pandas library
import pandas as pd

In [2]:
# Import the dataset into a dataframe
spam_data = pd.read_csv('SMSSpamCollection', sep='\t', header=None,
                       names = ['Label','SMS'])

In [3]:
# explore the set
spam_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [4]:
# see how many messages are classified as spam vs not
spamcounts = spam_data['Label'].value_counts()
print(spamcounts)

ham     4825
spam     747
Name: Label, dtype: int64


In [5]:
# report percent of spam messages
spam_percent = float(100*spamcounts[['spam']]/spam_data['Label'].count())
print('The set is {:.2f}% spam'.format(spam_percent))

The set is 13.41% spam


## Determining training and testing sets
We will use an 80:20 train:test split for our classifier. We will start by randomizing the dataset. For the purposes of this project we will use a random_state of 1 so that you can reproduce our results.

In [6]:
# randomize the dataset
random_spam_data = spam_data.sample(frac=1, random_state=1)

# split the data into a training and test set
train = random_spam_data.head(4458).reset_index()
test = random_spam_data.tail(1114).reset_index()

In [7]:
# report percentage of spam data in the training set
train_vals = train['Label'].value_counts()
train_spam_per = float(100*train_vals[['spam']]/train_vals.sum())
print('The training set is {:.2f}% spam'.format(train_spam_per))

The training set is 13.46% spam


In [8]:
# report percentage of spam data in the test set
test_vals = test['Label'].value_counts()
test_spam_per = float(100*test_vals[['spam']]/test_vals.sum())
print('The test set is {:.2f}% spam'.format(test_spam_per))

The test set is 13.20% spam


The proportion of spam in each of our samples closely match the spam proportion in the larger set, so we can consider each sample representative of the full dataset. 

## Cleaning the data
To implement our algorithm we will need to convert the data into a format that is easier to parse. We are going to convert the "SMS" column to a series of columns, each representing one of the words in the overall vocavulary of the set. For each word (column) and message (row), the data point will indicate the number of times that the word appears in the message.
To perform this operation we will also need to standardize the messages by removing punctuation and converting to lowercase.

In [9]:
# check head of original set
train.head()

,index,Label,SMS
0,1078,ham,"Yep, by the pretty sculpture"
1,4028,ham,"Yes, princess. Are you going to make me moan?"
2,958,ham,Welp apparently he retired
3,4642,ham,Havent.
4,4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
# Remove all punctuation and convert to lowercase
train['SMS'] = train['SMS'].str.replace(r'\W',' ').str.lower()
# check new output
train.head()

,index,Label,SMS
0,1078,ham,yep by the pretty sculpture
1,4028,ham,yes princess are you going to make me moan
2,958,ham,welp apparently he retired
3,4642,ham,havent
4,4674,ham,i forgot 2 ask ü all smth there s a card on ...


It seems that our operation worked. Now, we need to generate a vocabulary list which will store all the unique words encountered in the training set.

In [11]:
# initialize list for training set vocabulary
vocabulary = []

# transform training set SMS column into list of strings
train['SMS'] = train['SMS'].str.split()

# iterate through each message and store words if they have not already been stored
for sms in train['SMS']:
    for word in sms:
        if word not in vocabulary:
            vocabulary.append(word)

In [12]:
# initialize dictionary that stores the word counts of each sms. Each entry will
# start with a value of zero
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}

# iterate through the sms messages and their corresponding indices
for index, sms in enumerate(train['SMS']):
    # iterate through each word in the SMS and increment the corresponding 
    # dictionary value by 1
    for word in sms:
        word_counts_per_sms[word][index] += 1

# transform our word counter into a dataframe
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)

# concatenate Dfs
train_clean = pd.concat([train, word_counts_per_sms], axis=1)

In [13]:
# check new output
train_clean.head()

,index,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,1078,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4028,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,958,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4642,ham,[havent],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4674,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Computing constants
Now that the data is clean,we can begin to implement the algorithm. Before we begin, we need to compute some overall probabilities (constants) for our training set:
- P(Spam): the probability that a message is spam
- P(Ham): the probability that a message is ham
- Nspam: the number of words in all the spam messages
- Nham: the number of words in all the ham messages
- Nvocabulary: the number of words in the vocabulary

In [14]:
# Compute P(spam) by dividing the number of spam messages by the total
p_spam = train_clean['Label'][train_clean['Label'] == 'spam'].count()/train_clean['Label'].count()

# compute p_ham from p_spam
p_ham = 1 - p_spam

In [15]:
# compute N(spam)
n_spam = train_clean[train_clean['Label']=='spam'].iloc[:,3:].sum(axis=1).sum(axis=0)

In [16]:
# compute N(ham)
n_ham = train_clean[train_clean['Label']=='ham'].iloc[:,3:].sum(axis=1).sum(axis=0)

In [17]:
# Initiate variable alpha
alpha = 1

In [18]:
# compute N(vocabulary)
n_vocabulary = len(vocabulary)

## Computing parameters
Naive Bayes classifiers are remarkably fast because much of the computational work is done before we even begin classification. Before we run the classifier, we need to compute various parameters relating the probabilities of encountering each word given that a message is spam or ham.

In [19]:
# Initialize two dictionaries, one for spam and one for ham, where each key-value
# pair is a unique word from our vocabulary and the value is 0
spam_dict = {unique_word: [0] for unique_word in vocabulary}
ham_dict = {unique_word: [0] for unique_word in vocabulary}

In [20]:
# isolate spam and ham messages from training set into two different dataframes
spam = train_clean[train_clean['Label'] == 'spam']
ham = train_clean[train_clean['Label'] == 'ham']

In [21]:
# iterate over the vocabulary and compute the probability of each word given
# spam and the probability of each word given ham. add the probabilities to their
# respective dictionaries
for word in vocabulary:
    n_wi_spam = spam[word].sum()
    p_wi_spam = (n_wi_spam + alpha)/(n_spam + alpha * n_vocabulary)
    n_wi_ham = ham[word].sum()
    p_wi_ham = (n_wi_ham + alpha)/(n_ham + alpha * n_vocabulary)
    spam_dict[word] = p_wi_spam
    ham_dict[word] = p_wi_ham

## Building the classification function
Now that we have our parameters in place, we can build a function which parses messages from user input and classifies them. Remember that for each message we will need to perform a similar data cleaning operation that we did for our training set, standardizing the string input and splitting it into its component parts. From each word in the string, we can reference our vocabulary and parameters to compute the probability that the message is spam or ham, and output our classification.

In [22]:
# import regex so we can parse the input string
import re

# function classify takes an SMS message as a string and outputs either "Spam",
# "Ham", or a request for human help depending on the result of a Naive Bayes
# classifier
def classify(message):
    
    # Convert message into list of strings with all lowercase characters
    message = re.sub('\W', ' ', message).lower().split()
    
    # compute the probability that the message is spam and the probability of ham given the list of words
    # the formula to compute this is to multiply the probability of spam by
    # the sum of the probabilities of each word given spam.
    
    # initialize variables to report spam/ham probabilityies
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # iterate through each word in the message
    for word in message:
        # check if the word is in the spam dictionary
        if word in spam_dict:
            # if so, multiply our spam vairable by the value
            p_spam_given_message *= spam_dict[word]
            
        # check if the word is in the ham dictionary
        if word in ham_dict:
            # if so, multiply our ham variable by the value
            p_ham_given_message *= ham_dict[word]
    
    # compare the probabilities and report based on result
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
        
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

## Checking the classifier
Let's start with two new messages of our own design:

In [23]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [24]:
classify('Sounds good, Tom, then see u there')

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


Unfortunately, testing a classifier against a sample of two is not very robust. Now, it's time to test our classification against the test set that we reserved earlier. We'll modify our classification function from above so that it returns the labels instead of printing them. This way we can feed the results into a new column.

In [25]:
# import regex so we can parse the input string
import re

# function classify takes an SMS message as a string and outputs either "Spam",
# "Ham", or a request for human help depending on the result of a Naive Bayes
# classifier
def classify_test_set(message):
    
    # Convert message into list of strings with all lowercase characters
    message = re.sub('\W', ' ', message).lower().split()
    
    # compute the probability that the message is spam and the probability of ham given the list of words
    # the formula to compute this is to multiply the probability of spam by
    # the sum of the probabilities of each word given spam.
    
    # initialize variables to report spam/ham probabilityies
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # iterate through each word in the message
    for word in message:
        # check if the word is in the spam dictionary
        if word in spam_dict:
            # if so, multiply our spam vairable by the value
            p_spam_given_message *= spam_dict[word]
            
        # check if the word is in the ham dictionary
        if word in ham_dict:
            # if so, multiply our ham variable by the value
            p_ham_given_message *= ham_dict[word]
        
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [26]:
# Generate new column with predictions for test data
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,index,Label,SMS,predicted
0,2131,ham,Later i guess. I needa do mcat study too.,ham
1,3418,ham,But i haf enuff space got like 4 mb...,ham
2,3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,5393,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can measure the accuracy by comparing the predictions to the true values.

In [27]:
(test['predicted'] == test['Label']).sum() / test['Label'].count()

0.9874326750448833